*Este notebook apresenta a resolução para a avaliação proposta como parte do processo seletivo referente a vaga  [QCDGVU] SENAI | Cientista de Dados I - Inteligência de Mercado - São Paulo - SP | Nº 407-2024, desenvolvido pelo candidato Thiago de Souza Fontanari Maciel. Desenvolvido com Visual Studio Code*

#### Questão 1.1 - Criar Estrutura de Diretórios e Arquivos

Para a resolução do problema proposto, foi adotada a utilização do Python e suas bibliotecas como ferramentas principais. Foi determinado, também, trabalhar com arquivos tipo CSV (Comma-Separated Values) na base de dados local, pois este formato favorece a sua manipulação durante consultas e atualizações, uma vez que leituras e escritas sobre esse tipo de arquivo tendem a ser mais rápidas, e possibilita a utilização de bibliotecas extremamente eficientes como o Pandas, que será a principal ferramenta utilizada nesta resolução.

Quanto à estrutura, optou-se por criar diretórios na raiz do projeto com os nomes dos países de origem, e subdiretórios nomeados de acordo com o país de destino, estes últimos recebem os arquivos com os dados de report mensal e anual, no formato 'AAAA_M.csv' e 'AAAA_A.csv', respectivamente, onde a seção 'AAAA' é substituída pelo ano. 

De acordo com as informações fornecidas no enunciado, as consultas da API seguem o padrão *(pais_origem, pais_destino, ano)*, porém não é especificado o formato do retorno que a API fornece. Sendo assim, será adotado que retorno será um arquivo .csv, através do atributo *response.text* do objeto gerado pelo método *get*. O arquivo de report mensal será composto pelas colunas MES, IMP e EXP (representando, respectivamente o mês, a quantidade de importações e a quantidade de exportações) e 12 linhas, sendo uma para cada mês do ano. O arquivo de report anual conterá apenas as colunas 'IMP' e 'EXP' e uma linha para os dados. A primeira coluna do report mensal será utilizada como índice para realização de consultas. 

Será assumido, também, que os mantenedores dos dados na API podem fornecer os arquivos com dados faltantes, seja porque ainda não foram divulgados pelos países ou porque ainda não foram inseridos na base de dados. Esses dados ausentes serão tratados no item 1.2 desta solução, onde será inserido o valor '0' nesses casos.

Abaixo, é apresentada a estrutura de diretórios mais detalhadamente:

    comercio_exterior/
    |-- BRA/
    |   |-- CHN/
    |   |   |-- 2019_M.csv
    |   |   |-- 2019_A.csv    
    |   |   |-- 2020_M.csv
    |   |   |-- 2020_A.csv    
    |   |   |-- 2021_M.csv
    |   |   |-- 2021_A.csv    
    |   |-- USA/
    |   |   |-- 2019_M.csv
    |   |   |-- 2019_A.csv    
    |   |   |-- 2020_M.csv
    |   |   |-- 2020_A.csv    
    |   |   |-- 2021_M.csv
    |   |   |-- 2021_A.csv    
    ...

#### Questão 1.2 - Integrar os Dados de três anos (2019,2020 e 2021) a partir da API

Com base nas informações fornecidas no enunciado, a estrutura definida no item 1.1 e o formato de retorno da API também definido no item 1.1, são necessários os seguintes passos para abastecer a base de dados local:

    - Desenvolver a solicitação GET ao endpoint da API;
    - Receber e tratar o retorno da solicitação;
    - Armazenar as informações obtidas na estrutura criada no item anterior;

 Abaixo é definida uma função que realiza a consulta na API dos anos solicitados e salva em seus respectivos arquivos de destino (a lista contendo os países foi obtida a partir dos códigos de 3 letras definidos pela ISO 3166-1 alfa-3, disponível em https://pt.wikipedia.org/wiki/ISO_3166-1_alfa-3). Também conforme orientação do enunciado, não será explicitada a função *get* que faz a consulta na API:

In [ ]:
# Importando as bibliotecas utilizadas
import os
import requests # Na aplicação real, será necessário para realizar o get
import pandas as pd
from io import StringIO # Será utilizado para ler a string retornada pelo get

def get(pais_origem, pais_destino, ano):
    # A função não foi explicitada, serve apenas para simular a consulta da API
    return response.text # Retorno esperado da API é um CSV em formato de string

paises = ['AFG', 'ZAF', 'ALB', 'DEU', 'AND', 'AGO', 'AIA', 'ATA', 'ATG', 'ANT', 'SAU',
          'DZA', 'ARG', 'ARM', 'ABW', 'AUS', 'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BRB',
          'BLR', 'BEL', 'BLZ', 'BEN', 'BMU', 'BOL', 'BIH', 'BWA', 'BRA', 'BRN', 'BGR',
          'BFA', 'BDI', 'BTN', 'CPV', 'CMR', 'KHM', 'CAN', 'KAZ', 'TCD', 'CHL', 'CHN',
          'CYP', 'SGP', 'COL', 'COG', 'PRK', 'KOR', 'CIV', 'CRI', 'HRV', 'CUB', 'DNK',
          'DJI', 'DMA', 'EGY', 'SLV', 'ARE', 'ECU', 'ERI', 'SVK', 'SVN', 'ESP', 'USA', 
          'EST', 'ETH', 'FJI', 'PHL', 'FIN', 'FRA', 'GAB', 'GMB', 'GHA', 'GEO', 'GIB', 
          'GBR', 'GRD', 'GRC', 'GRL', 'GLP', 'GUM', 'GTM', 'GGY', 'GUY', 'GUF', 'GIN', 
          'GNQ', 'GNB', 'HTI', 'NLD', 'HND', 'HKG', 'HUN', 'YEM', 'BVT', 'IMN', 'CXR', 
          'PCN', 'REU', 'ALA', 'CYM', 'CCK', 'COM', 'COK', 'FRO', 'FLK', 'SGS', 'HMD', 
          'MNP', 'MHL', 'UMI', 'NFK', 'SYC', 'SLB', 'SJM', 'TKL', 'TCA', 'VIR', 'VGB', 
          'WLF', 'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'ISL', 'ISR', 'ITA', 'JAM', 'JPN', 
          'JEY', 'JOR', 'KEN', 'KIR', 'KWT', 'LAO', 'LVA', 'LSO', 'LBN', 'LBR', 'LBY', 
          'LIE', 'LTU', 'LUX', 'MAC', 'MKD', 'MDG', 'MYS', 'MWI', 'MDV', 'MLI', 'MLT', 
          'MAR', 'MTQ', 'MUS', 'MRT', 'MYT', 'MEX', 'FSM', 'MOZ', 'MDA', 'MCO', 'MNG', 
          'MNE', 'MSR', 'MMR', 'NAM', 'NRU', 'NPL', 'NIC', 'NER', 'NGA', 'NIU', 'NOR', 
          'NCL', 'NZL', 'OMN', 'PLW', 'PAN', 'PNG', 'PAK', 'PRY', 'PER', 'PYF', 'POL', 
          'PRI', 'PRT', 'QAT', 'KGZ', 'CAF', 'COD', 'DOM', 'CZE', 'ROM', 'RWA', 'RUS', 
          'ESH', 'VCT', 'ASM', 'WSM', 'SMR', 'SHN', 'LCA', 'BLM', 'KNA', 'MAF', 'STP', 
          'SEN', 'SLE', 'SRB', 'SYR', 'SOM', 'LKA', 'SPM', 'SWZ', 'SDN', 'SWE', 'CHE', 
          'SUR', 'TJK', 'THA', 'TWN', 'TZA', 'IOT', 'ATF', 'PSE', 'TMP', 'TGO', 'TON', 
          'TTO', 'TUN', 'TKM', 'TUR', 'TUV', 'UKR', 'UGA', 'URY', 'UZB', 'VUT', 'VAT', 
          'VEN', 'VNM', 'ZMB', 'ZWE', 'WLD']

anos = ['2019', '2020', '2021']

# Aqui será realizada a iteração sobre a lista de países e anos de interesse
# e a consulta na API

for pais_origem in paises:
    for pais_destino in paises:
        if pais_origem != pais_destino and pais_origem != 'WLD':
            for ano in anos: 

                # Realiza a consulta
                dados_csv = get(pais_origem, pais_destino, ano)

                # Converte a string CSV para DataFrame de Pandas, lendo a string 
                # retornada pelo get através do método StrinIO
                data_frame = pd.read_csv(StringIO(dados_csv))
                data_frame.fillna(0, inplace=True) # Preenchendo valores vazios com zero
                data_frame.set_index('MES', inplace=True) # Transformando a coluna 'MES' em indice do dataframe, para facilitar a consulta no futuro

                # Cria o diretório para salvar o arquivo, caso não exista
                diretorio = f'comercio_exterior/{pais_origem}/{pais_destino}'
                if not os.path.exists(diretorio):
                    os.makedirs(diretorio)

                # Salva o arquivo no diretorio de destino
                data_frame.to_csv(f'{diretorio}{ano}_M.csv', index=True, encoding='utf-8')

print('Base de dados atualizadda com sucesso!') # Mensagem de sucesso para o usuário



#### Questão 1.3 - Qual a quantidade de soja que o Mundo importou do Brasil em 2020?

De acordo com a estrutura de diretórios e arquivos criada nessa sulução, a resposta estará no arquivo 'comercio_exterior/BRA/WLD/2020_A.csv'.

#### Questão 1.4 - Realização de Consultas para Alimentar o Painel Informativo

Para atender as solicitações do enunciado, deverão ser observados os seguintes passos:

    - Desenvolver uma função que realiza consulta na base de dados local e retorna o report mensal ou anual, dependendo do parâmetro informado (A ou M);
    
    - Criar uma rotina de atualização da base de dados local a partir de dados consumidos da API;

Para atender esses requisitos, abaixo será implementada uma rotina de atualização com a biblioteca schedule do Python.


In [ ]:
import pandas as pd
import schedule
import time
from datetime import datetime, timedelta

# Função para consultar a base de dados local

def report(pais_origem, pais_destino, ano, tipo):
    
    '''
    Primeiro, será selecionado o arquivo para geração do report de acordo com o tipo da 
    consulta realizada (report mensal ou anual). Os valores retornados serão armazenados 
    em um dicionário, que pode ser utilizado posteriormente pelo painel informativo. O
    dicionário armazenará os dados no formato {Ano: (Importação, Exportação)}.
    '''
    dados_do_report = {} # Inicia o dicionário para armazenar o report
    
    # Verifica qual o tipo de consulta
    if tipo == 'M':
        arquivo = f'comercio_exterior/{pais_origem}/{pais_destino}/{ano}_M.csv'

    else:
        arquivo = f'comercio_exterior/{pais_origem}/{pais_destino}/{ano}_A.csv'

    # Carrega o arquivo em um DataFrame Pandas
    data_frame = pd.read_csv(arquivo)

    # Preenche o dicionário com os valores do arquivo
    for mes, linha in data_frame.iterrows():
        dados_do_report[mes] = (linha['IMP'], linha['EXP'])

    return dados_do_report  

# Rotina de atualização da base de dados local

def atualiza_Base():
    
    '''
    Função que realiza a consulta na API e atualiza os dados locais. De forma semelhante
    à função implementada no item 1.2, essa função realiza uma requisição à API e salva
    os dados recebidos na base de dados local.

    '''
    # Lista de países para atualizar
    paises = ['AFG', 'ZAF', 'ALB', 'DEU', 'AND', 'AGO', 'AIA', 'ATA', 'ATG', 'ANT', 'SAU',
          'DZA', 'ARG', 'ARM', 'ABW', 'AUS', 'AUT', 'AZE', 'BHS', 'BHR', 'BGD', 'BRB',
          'BLR', 'BEL', 'BLZ', 'BEN', 'BMU', 'BOL', 'BIH', 'BWA', 'BRA', 'BRN', 'BGR',
          'BFA', 'BDI', 'BTN', 'CPV', 'CMR', 'KHM', 'CAN', 'KAZ', 'TCD', 'CHL', 'CHN',
          'CYP', 'SGP', 'COL', 'COG', 'PRK', 'KOR', 'CIV', 'CRI', 'HRV', 'CUB', 'DNK',
          'DJI', 'DMA', 'EGY', 'SLV', 'ARE', 'ECU', 'ERI', 'SVK', 'SVN', 'ESP', 'USA', 
          'EST', 'ETH', 'FJI', 'PHL', 'FIN', 'FRA', 'GAB', 'GMB', 'GHA', 'GEO', 'GIB', 
          'GBR', 'GRD', 'GRC', 'GRL', 'GLP', 'GUM', 'GTM', 'GGY', 'GUY', 'GUF', 'GIN', 
          'GNQ', 'GNB', 'HTI', 'NLD', 'HND', 'HKG', 'HUN', 'YEM', 'BVT', 'IMN', 'CXR', 
          'PCN', 'REU', 'ALA', 'CYM', 'CCK', 'COM', 'COK', 'FRO', 'FLK', 'SGS', 'HMD', 
          'MNP', 'MHL', 'UMI', 'NFK', 'SYC', 'SLB', 'SJM', 'TKL', 'TCA', 'VIR', 'VGB', 
          'WLF', 'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'ISL', 'ISR', 'ITA', 'JAM', 'JPN', 
          'JEY', 'JOR', 'KEN', 'KIR', 'KWT', 'LAO', 'LVA', 'LSO', 'LBN', 'LBR', 'LBY', 
          'LIE', 'LTU', 'LUX', 'MAC', 'MKD', 'MDG', 'MYS', 'MWI', 'MDV', 'MLI', 'MLT', 
          'MAR', 'MTQ', 'MUS', 'MRT', 'MYT', 'MEX', 'FSM', 'MOZ', 'MDA', 'MCO', 'MNG', 
          'MNE', 'MSR', 'MMR', 'NAM', 'NRU', 'NPL', 'NIC', 'NER', 'NGA', 'NIU', 'NOR', 
          'NCL', 'NZL', 'OMN', 'PLW', 'PAN', 'PNG', 'PAK', 'PRY', 'PER', 'PYF', 'POL', 
          'PRI', 'PRT', 'QAT', 'KGZ', 'CAF', 'COD', 'DOM', 'CZE', 'ROM', 'RWA', 'RUS', 
          'ESH', 'VCT', 'ASM', 'WSM', 'SMR', 'SHN', 'LCA', 'BLM', 'KNA', 'MAF', 'STP', 
          'SEN', 'SLE', 'SRB', 'SYR', 'SOM', 'LKA', 'SPM', 'SWZ', 'SDN', 'SWE', 'CHE', 
          'SUR', 'TJK', 'THA', 'TWN', 'TZA', 'IOT', 'ATF', 'PSE', 'TMP', 'TGO', 'TON', 
          'TTO', 'TUN', 'TKM', 'TUR', 'TUV', 'UKR', 'UGA', 'URY', 'UZB', 'VUT', 'VAT', 
          'VEN', 'VNM', 'ZMB', 'ZWE', 'WLD']
    
    for pais_origem in paises:
        for pais_destino in paises:
            if pais_origem != pais_destino and pais_origem != 'WLD':
                for ano in anos: 

                    # Realiza a consulta
                    dados_csv = get(pais_origem, pais_destino, ano)

                    # Converte a string CSV para DataFrame de Pandas, lendo a string 
                    # retornada pelo get através do método StrinIO
                    data_frame = pd.read_csv(StringIO(dados_csv))
                    data_frame.fillna(0, inplace=True) # Preenchendo valores vazios com zero
                    data_frame.set_index('MES', inplace=True) # Transformando a coluna 'MES' em indice do dataframe, para facilitar a consulta no futuro

                    # Cria o diretório para salvar o arquivo, caso não exista
                    diretorio = f'comercio_exterior/{pais_origem}/{pais_destino}'
                    if not os.path.exists(diretorio):
                        os.makedirs(diretorio)

                    # Salva o arquivo no diretorio de destino
                    data_frame.to_csv(f'{diretorio}{ano}_M.csv', index=True, encoding='utf-8')

# Agendamento das atualizações

'''
    Na seção abaixo, foi implementada a rotina de atualização da base dados, de forma 
    automática, todo primeiro dia do mês às 09:00AM. A rotina calcula automaticamente
    o próximo dia que a atualização deve ser executada e permanece verificando se a
    data chegou de forma contínua.

'''

def rotina_De_Atualização():

    # Obtem a data atuaç
    hoje = datetime.now()

    # Calcula o primeiro dia do próximo mês
    if hoje.month == 12:
        proximo_mes = datetime(hoje.year + 1, 1, 1, 9, 0)
    else:
        proximo_mes = datetime(hoje.year, hoje.month + 1, 1, 9, 0)
    
    # Agenda a próxima execução para a data determinada acima
    schedule.every().day.at(proximo_mes.strftime("%H:%M")).do(atualiza_Base).tag('mensal')

# Simulando a inicialização do agendamento do próximo mês
rotina_De_Atualização()

# Função para verificar a data e executar a atualização automaticamente
def roda_agendador():
    while True:
        schedule.run_pending()
        time.sleep(1)  # Para pupar o uso de CPU, foi implementada esta pausa no
                       # no loop infinito, o valor 1 é meramente ilustrativo
                       
        # Verifica se a função foi executada e agenda a próxima execução
        if not schedule.get_jobs('mensal'):
            agendar_proximo_mes()

# Simulando a execução do agendador
roda_agendador()



#### Questão 2

Para a resolução do problema proposto, sugere-se a adoção dos seguintes passos:

1 -> Determinar a estrutura do ambiente de trabalho, onde deverão ser definidas as ferramentas utilizadas na manipulação e processamento dos arquivos baixados (linguagens, bibliotecas, etc), bem como a estrutura de armazenamento local. Um ponto de antenção especial será o formato que os arquivos deverão ter após o processamento, que deve respeitar as particularidades impostas pela forma como serão consumidos da base de dados local. Para exemplificação, nos próximos itens, serão consideradas como ferramentas adotadas a linguagem Python e suas bibliotecas.

2 -> Uma vez que as atualizações são realizadas mensalmente, o sistema deve possuir uma rotina que verifique a disponibilização de arquivos atualizados e faça a atualização dos dados locais, de maneira reccorrente. Considerando que as atualizações podem ocorrer em dias diferentes do mês, sugere-se uma rotina que verifica o site diariamente, até que ocorra uma atualização (através do atributo 'Last modified' dos arquivos). Ao encontrar a existência de atualizações, o sistema executa a etapa de atualização da base local e reagenda a verificação no site para iniciar a partir do primeiro dia do próximo mês. Tais mecanismos podem ser implementados utilizando ferramentas disponíveis nas bibliotecas 'schedule' e 'time' do Python ou através de shell scripts, por exemplo .

3 -> Implementação dos mecanismos de download dos arquivos novos do site. É possível realizar essa implementação através da biblioteca 'requests'. Para atenuar os problemas com a instabilidade do site, se faz necessária a implementação de loops de retentativas de download, em caso de falha, através de 'retries'. Após o download, realizar verificações de integridade dos arquivos por checksums ou através do 'zipfile'.

4 -> Caso seja identificado um arquivo corrompido através dos métodos implementados no passo anterior, é necessário que o sistema faça o download novamente.

5 -> Após a realização bem-sucedida dos downloads, realizar a extração dos arquivos, também com o 'zipfile'.

6 -> Realizar o empilhamento dos arquivos, utilizando o Pandas. Carregar os arquivos extraídos em DataFrames e concatená-los com o método pandas.concat. Ressalta-se que a maneira exata de como os arquivos serão concatenados deve ser definida na primeira etapa desse processo de trabalho.

7 -> Após o download e tratamento dos arquivos, salvar o resultado na base de dados local. A forma de armazenamento deve ser definida na primeira etapa, e pode ser, por exemplo, armazenamento em arquivos CSV.

8 -> Garantir a existência de um mecanismo de backups e recuperação, salvando os dados tratados em armazenamentos redundantes, afim de evitar perdas. 

9 -> Criar um mecanismo de geração e tratamento de logs, onde os eventos podem ser reportados por uma API de mensagens via e-mail, por exemplo. É importante que sejam reportadas falhas de download, indisponibilidade no site, erros na persistência de dados no armazenamento local, etc. Os logs gerados, especialmente os que reportam erros, devem ser claros e de fácil interpretação, afim de viabilizar uma intervenção mais ágil na resolução de problemas.

#### Questão 3

A consulta SQL que reponde a questão é a seguinte:

SELECT
    e.employee_id,
    e.employee_name,
    d1.department_name AS first_department,
    d2.department_name AS current_department,
    COUNT(DISTINCT edh.department_id) AS department_count
FROM
    employees e
JOIN
    employee_department_history edh ON e.employee_id = edh.employee_id
JOIN
    departments d1 ON d1.department_id = (
        SELECT department_id
        FROM employee_department_history edh_sub
        WHERE edh_sub.employee_id = e.employee_id
        ORDER BY start_date ASC
        LIMIT 1
    )
JOIN
    departments d2 ON d2.department_id = (
        SELECT department_id
        FROM employee_department_history edh_sub
        WHERE edh_sub.employee_id = e.employee_id
          AND edh_sub.end_date IS NULL
        LIMIT 1
    )
GROUP BY
    e.employee_id,
    e.employee_name,
    d1.department_name,
    d2.department_name


#### Questão 4

Dada a situação do enunciado, e considerando a impossibilidade de alterações diretas sobre o arquivo .parquet, uma estratégia interessante é desenvolver uma API que faça cacheamento com Redis das consultas mais frequentes. É apresentada uma base de dados com uma baixa taxa de atualização (alta taxa de leitura e baixa taxa de escrita), o que é interessante no uso de cache pois torna a vida útil dos dados em cache maior. Para o desenvolvimento da API, temos como uma boa opção a utilização do framework Flask, que permite o desenvolvimento de API's leves e com bom desempenho, características que se enquadram em uma boa suloção para o problema proposto.

##### Questão 5

Ao obter os resultados inesperados/insatisfatórios relatados no enunciado, a princípio, pode-se realizar uma análise exploratória do dados, afim de identificar dados faltantes, tratar a possível existência de outliers e verificar se a normalização dos dados está correta para a ingestão do modelo. Problemas relacionados à esses aspectos em uma base de dados podem comprometer o resultado retornado por um modelo.

Também é possível atuar no aprimoramento do modelo e na sua implementação. A tentativa de uso de modelos diferentes pode trazer resultados mais satisfatórios, bem como utilizar validação cruzada para ajustar os hiperparâmetros e evitar overfitting do modelo. Outro ponto de atenção é a implementação do código em si, onde é possível encontrar janelas de melhoria para otimização da eficiência computacional e complexidade de tempo de execução.

#### Questão 6

A alternatia que apresenta a resposta correta é: Alternativa D.